# Spam Detection using LSTM neural network

In this notebook, I will implement a recurrent neural network that perform spam detection. Using an RNN/LSTM rather than a feedforward or logistic regression is more accurate since we can include information about sequence of words.

In [139]:
import tensorflow as tf
import numpy as np
import os

# Load data

In [140]:
with open('Assets/SMSSpamCollection', 'r') as f:
    data = f.read()
    
data[:300]

"ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\nham\tOk lar... Joking wif u oni...\nspam\tFree entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 084528100"

# Preprocessing

In [141]:
# Remove punctuation and lowercase
from string import punctuation
all_text = ''.join([c for c in data if c not in punctuation])
all_text = all_text.lower()

# split label and text of each line.
messages = all_text.split('\n')
messages = [x.split('\t') for x in messages if len(x)>=1]
[labels, texts] = np.array([list(x) for x in zip(*messages)])

In [142]:
labels = np.array([1 if each == 'spam' else 0 for each in labels])

In [143]:
labels

array([0, 0, 1, ..., 0, 0, 0])

In [144]:
# All words
all_text = ' '.join(texts)
words = all_text.split()
words[:20]

['go',
 'until',
 'jurong',
 'point',
 'crazy',
 'available',
 'only',
 'in',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet',
 'cine',
 'there',
 'got',
 'amore',
 'wat']

# Encoding the words

In [145]:
from collections import Counter
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse = True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab)}

In [146]:
vocab_to_int

{'butting': 4393,
 'wings': 4394,
 'easily': 2907,
 'satã¼': 4412,
 'knw': 647,
 'audition': 2345,
 'fourth': 6139,
 'kegger': 2908,
 'taxless': 4398,
 'mjzgroup': 4429,
 'wid': 547,
 'chuck': 4400,
 'tops': 4401,
 'birthday': 328,
 'yep': 957,
 'opted': 4402,
 'embarassing': 4403,
 '4the': 4405,
 '6hrs': 1848,
 'familiar': 4407,
 'sonyericsson': 1600,
 'marsms': 4408,
 'placement': 4161,
 'dai': 1407,
 '01223585236': 4410,
 'fuckin': 2229,
 'cribbs': 4411,
 'forgets': 2909,
 'participate': 7054,
 'ktv': 4413,
 'smacks': 4415,
 'bring': 460,
 'winterstone': 8401,
 'kall': 4416,
 'childporn': 4417,
 'haf': 507,
 'txtin': 1843,
 'anot': 2910,
 '7634': 4419,
 'tick': 1408,
 'auto': 2230,
 '08712402902': 4420,
 'bw': 7055,
 'lotto': 4422,
 'kit': 4423,
 'feelin': 2938,
 'help08718728876': 3175,
 'nordstrom': 5256,
 'across': 958,
 'ignorant': 4427,
 'surya': 6147,
 'box420': 4399,
 'teaches': 1845,
 '4u': 1140,
 'failing': 4430,
 'kodthini': 4431,
 'spelling': 6023,
 'fumbling': 4432,
 'te

In [147]:
# Convert the reviews to integers, same shape as reviews list, but with integers
text_ints = []
for each in texts:
    text_ints.append([vocab_to_int[word] for word in each.split()])

In [148]:
text_ints[3]

[5, 235, 142, 23, 357, 2955, 5, 162, 143, 59, 142]

In [149]:
from collections import Counter
text_lens = Counter([len(x) for x in text_ints])
print("Zero-length text: {}".format(text_lens[0]))
print("Maximum text length: {}".format(max(text_lens)))

Zero-length text: 2
Maximum text length: 171


In [150]:
non_zero_idx = [ii for ii, texts in enumerate(text_ints) if len(texts) != 0]
len(non_zero_idx)

5572

In [151]:
len(texts)

5574

In [152]:
# fillter out that review with 0 length
text_ints = [text_ints[ii] for  ii in non_zero_idx]
labels = np.array([labels[ii] for ii in non_zero_idx])

In [153]:
seq_len = 150
features = np.zeros((len(text_ints), seq_len), dtype=int)
for i, row in enumerate(text_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]

In [154]:
features[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,   45,  447,
       8667,  840,  747,  692,   63,    8, 1362,   88,  121,  360, 1288,
        152, 3559, 1262,   67,   56, 8253,  137])

# Training, Validation, Test

In [155]:
split_frac = 0.8
split_idx = int(len(features)*0.8)
train_x, val_x = features[:split_idx], features[split_idx:]
train_y, val_y = labels[:split_idx], labels[split_idx:]

test_idx = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(4457, 150) 
Validation set: 	(557, 150) 
Test set: 		(558, 150)


# Build the graph

In [156]:
lstm_size = 256
lstm_layers = 1
batch_size = 250
learning_rate = 0.001
drop_out = 0.5
epochs = 5

In [157]:
n_words = len(vocab_to_int)

# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, [None, None], name='inputs')
    labels_ = tf.placeholder(tf.int32, [None, None], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [158]:
# Size of the embedding vectors (number of units in the embedding layer)
embed_size = 300 

with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

In [159]:
with graph.as_default():
    # Your basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

In [160]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed,
                                             initial_state=initial_state)

In [161]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels_, predictions)
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [162]:
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [163]:
def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [164]:
with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob: drop_out,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/sentiment.ckpt")

Epoch: 0/5 Iteration: 5 Train loss: 0.108
Epoch: 0/5 Iteration: 10 Train loss: 0.083
Epoch: 0/5 Iteration: 15 Train loss: 0.047
Epoch: 1/5 Iteration: 20 Train loss: 0.043
Epoch: 1/5 Iteration: 25 Train loss: 0.025
Val acc: 0.964
Epoch: 1/5 Iteration: 30 Train loss: 0.018
Epoch: 2/5 Iteration: 35 Train loss: 0.021
Epoch: 2/5 Iteration: 40 Train loss: 0.025
Epoch: 2/5 Iteration: 45 Train loss: 0.017
Epoch: 2/5 Iteration: 50 Train loss: 0.017
Val acc: 0.968
Epoch: 3/5 Iteration: 55 Train loss: 0.015
Epoch: 3/5 Iteration: 60 Train loss: 0.007
Epoch: 3/5 Iteration: 65 Train loss: 0.018
Epoch: 4/5 Iteration: 70 Train loss: 0.002
Epoch: 4/5 Iteration: 75 Train loss: 0.001
Val acc: 0.974
Epoch: 4/5 Iteration: 80 Train loss: 0.016
Epoch: 4/5 Iteration: 85 Train loss: 0.000


In [165]:
test_acc = []
with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

INFO:tensorflow:Restoring parameters from checkpoints\sentiment.ckpt
Test accuracy: 0.978
